GaussianNB<br>
GaussianNB<br>
	特征变量是连续变量，符合高斯分布<br>
MultinomialNB<br>
	特征变量是离散变量，符合多项分布<br>
		在文档分类中特征变量体现在一个单词出现的次数，或者是单词的TF-IDF 值等<br>
		TF-IDF 值是一个统计方法，用来评估某个词语对于一个文件集或文档库中的其中一份文件的重要程度<br>
			TF-IDF 实际上是两个词组 Term Frequency 和 Inverse Document Frequency 的总称，两者缩写为 TF 和 IDF，分别代表了词<br>频和逆向文档频率<br>
	sklearn TfidfVectorizer<br>
BernoulliNB<br>
	特征变量是布尔变量，符合 0/1 分布<br>

In [96]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
# TfidfVectorizer(stop_words=stop_words, token_pattern=token_pattern)


文档 1：this is the bayes document；
文档 2：this is the second second document；
文档 3：and the third one；
文档 4：is this the document

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vec = TfidfVectorizer()


In [98]:
documents = [
    'this is the bayes document',
    'this is the second second document',
    'and the third one',
    'is this the document'
]
tfidf_matrix = tfidf_vec.fit_transform(documents)


In [99]:
print('unique words:', tfidf_vec.get_feature_names())


('unique words:', [u'and', u'bayes', u'document', u'is', u'one', u'second', u'the', u'third', u'this'])


In [100]:
print('word IDs:', tfidf_vec.vocabulary_)


('word IDs:', {u'and': 0, u'third': 7, u'this': 8, u'is': 3, u'one': 4, u'second': 5, u'bayes': 1, u'the': 6, u'document': 2})


In [101]:
print('the tfidf value of words:', tfidf_matrix.toarray())


('the tfidf value of words:', array([[0.        , 0.63314609, 0.40412895, 0.40412895, 0.        ,
        0.        , 0.33040189, 0.        , 0.40412895],
       [0.        , 0.        , 0.27230147, 0.27230147, 0.        ,
        0.85322574, 0.22262429, 0.        , 0.27230147],
       [0.55280532, 0.        , 0.        , 0.        , 0.55280532,
        0.        , 0.28847675, 0.55280532, 0.        ],
       [0.        , 0.        , 0.52210862, 0.52210862, 0.        ,
        0.        , 0.42685801, 0.        , 0.52210862]]))


## Get Data
https://github.com/cystanford/text_classification


/home/ml/predictOptimizaitonStatus/src/product_reviews_1<br>
    
This folder contains annotated customer reviews of 5 products.

        1. digital camera: Canon G3
        2. digital camera: Nikon coolpix 4300
        3. celluar phone:  Nokia 6610
        4. mp3 player:     Creative Labs Nomad Jukebox Zen Xtra 40GB
        5. dvd player:     Apex AD2600 Progressive-scan DVD player

In [102]:
import os
import io
text_directory = './product_reviews_1'
text_list = os.listdir(text_directory)
print text_list


['Apex_AD2600_Progressive_scan_DVD player.txt', 'Canon_G3.txt', 'Creative_Labs_Nomad_Jukebox_Zen_Xtra_40GB.txt', 'Nikon_coolpix_4300.txt', 'Nokia_6610.txt', 'README.txt']


### Defining stemmer, which is used to clean up the text. Normally delete some unwanted terms

In [129]:

from nltk.stem.porter import *
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed
stemmer = PorterStemmer()

### Preprocess the texts

1. put all them into lower words
2. remove punctuation
3. tokenize
4. stem them
5. adding label to each input

In [130]:
import nltk #for english
import string
file_name = 'Apex_AD2600_Progressive_scan_DVD player.txt'
label_dict = {'Apex_AD2600_Progressive_scan_DVD player.txt': 'Apex',
             'Canon_G3.txt': 'Canon_G3',
             'Creative_Labs_Nomad_Jukebox_Zen_Xtra_40GB.txt':'Nomad',
             'Nikon_coolpix_4300.txt':'Nikon4300',
             'Nokia_6610.txt':'Nokia6610'}
data_set = []
for file_name in text_list:
    file_dir = text_directory+ '/' + file_name
    with open(file_dir,'r') as f:
        data = f.readlines()
        for line in data:
            if line[0] == '#':
                l = line.replace('#', '').replace('\n', '').replace('\w', '')
                lowers = l.lower()
                no_punctuation = lowers.translate(None, string.punctuation)
                # tokenize
                tokens = nltk.word_tokenize(no_punctuation)
#                 filtered = [w for w in tokens if not w in stopwords.words('english')]
                stemmed = stem_tokens(tokens, stemmer)
                data_set.append((' '.join(stemmed), label_dict[file_name]))


### Transforming to Dataframe to simplify the sklearn implementatons

In [105]:
import pandas as pd
labels = ['stemmed_review', 'label']
df = pd.DataFrame(data_set, columns=labels)

X = df['stemmed_review']
y = df['label']

### overview of the stats of the labels

In [106]:
y.describe()

count      2214
unique        5
top       Nomad
freq        995
Name: label, dtype: object

In [107]:
y.describe(include='all')

count      2214
unique        5
top       Nomad
freq        995
Name: label, dtype: object

In [108]:
y.value_counts()

Nomad        995
Apex         395
Canon_G3     358
Nokia6610    280
Nikon4300    186
Name: label, dtype: int64

<b>It seems there's some sort of imbalance between labels, such as the Nikon4300 counts small portion than others

## Shuffle and split dataset

In [134]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [135]:
X_train.head()

593     i do nt need to go into exhust review of thi c...
1023                                   thi is pretti cool
973                                               summari
782     you wo nt appreci the qualiti thi thing ha to ...
1934    i am a busi user who heavili depend on mobil s...
Name: stemmed_review, dtype: object

In [136]:
y_train.head()

593      Canon_G3
1023        Nomad
973         Nomad
782         Nomad
1934    Nokia6610
Name: label, dtype: object

We need to download the normal stopwords file from internet and read it via python

## Load stop words

Stop word is used to filter out the original text and calcualtion the text scores such as TF-IDF

In [112]:
# nltk.download('stopwords')
from nltk.corpus import stopwords
stopWords = set(stopwords.words('english'))

In [113]:
print('Train:')
print('input: ', X_train.shape)
print('label: ', y_train.shape)
print('Test:')
print('input: ', X_test.shape)
print('label: ', y_test.shape)

Train:
('input: ', (1483,))
('label: ', (1483,))
Test:
('input: ', (731,))
('label: ', (731,))


## calculate TF-IDF

We use this as input to bays's model, then bays model use it to calcualte the classification

TF-IDF 值是一个统计方法，用来评估某个词语对于一个文件集或文档库中的其中一份文件的重要程度

TF-IDF 实际上是两个词组 Term Frequency 和 Inverse Document Frequency 的总称，两者缩写为 TF 和 IDF，分别代表了词
频和逆向文档频率

这里 max_df 参数用来描述单词在文档中的最高出现率。假设 max_df=0.5，代表一个单词在 50% 的文档中都出现过了，那么它只携带了非常少的信息，因此就不作为分词统计。



In [137]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [138]:
vectorizer = TfidfVectorizer(stop_words=stopWords, max_df=0.5)
# train_vocabulary = tf.vocabulary_()
X_train_unpipe = vectorizer.fit_transform(X_train)

In [139]:
print(X_train_unpipe.shape)

(1483, 2355)


## Train model

In [140]:
# 多项式贝叶斯分类器
from sklearn.naive_bayes import MultinomialNB  
clf = MultinomialNB(alpha=0.0001).fit(X_train_unpipe, y_train)


In [141]:
print clf.get_params()


{'alpha': 0.0001, 'fit_prior': True, 'class_prior': None}


## Predict

In [142]:

X_test_unpipe = vectorizer.transform(X_test)
predicted_labels=clf.predict(X_test_unpipe)


## Model evaluation

In [143]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
print (metrics.accuracy_score(y_test, predicted_labels))
confusion_matrix(y_test, predicted_labels)

from sklearn.metrics import classification_report
target_names = label_dict.values()
print(classification_report(y_test, predicted_labels, target_names=target_names))

0.6265389876880985
              precision    recall  f1-score   support

       Nomad       0.67      0.40      0.50       140
    Canon_G3       0.63      0.53      0.57       125
   Nokia6610       0.44      0.38      0.41        63
   Nikon4300       0.67      0.44      0.53        85
        Apex       0.63      0.86      0.73       318

   micro avg       0.63      0.63      0.63       731
   macro avg       0.61      0.52      0.55       731
weighted avg       0.63      0.63      0.61       731



## Pipeline version, be careful of the naming while changing from unpiped version

In [144]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB  

pipe = []
# vectorizer = ('vectorizer', CountVectorizer())
# pipe.append(vectorizer)

vectorizer = ('vectorizer', TfidfVectorizer())
pipe.append(vectorizer)

p_clf =  ('clf', MultinomialNB(alpha=0.001))
pipe.append(p_clf)



pipe_clf = Pipeline(pipe)
pipe_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=u'l2', preprocessor=None, smooth_...e,
        vocabulary=None)), ('clf', MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True))])

In [145]:
predicted_labels = pipe_clf.predict(X_test)

In [146]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
print ('Accuracy: ', metrics.accuracy_score(y_test, predicted_labels))
confusion_matrix(y_test, predicted_labels)

from sklearn.metrics import classification_report
target_names = label_dict.values()
print('\n')
print(classification_report(y_test, predicted_labels, target_names=target_names))

('Accuracy: ', 0.6415868673050615)


              precision    recall  f1-score   support

       Nomad       0.68      0.40      0.50       140
    Canon_G3       0.66      0.56      0.61       125
   Nokia6610       0.51      0.33      0.40        63
   Nikon4300       0.75      0.45      0.56        85
        Apex       0.63      0.89      0.74       318

   micro avg       0.64      0.64      0.64       731
   macro avg       0.65      0.53      0.56       731
weighted avg       0.65      0.64      0.62       731

